# Duration Prediction

In [1]:
# Check python version
!python -V

Python 3.9.10


In [2]:
# Load dependencies
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer

import mlflow

from config import PATH_DATA_GT_JAN, PATH_DATA_GT_FEB
from src.load_data import load_data
from src.preprocess_data import preprocess

In [3]:
# Configure mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc_taxi_experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

In [4]:
# Load datasets
df_train = load_data(PATH_DATA_GT_JAN)
df_val = load_data(PATH_DATA_GT_FEB)

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
# Data preprocessing

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

# Select numerical and categorical variables
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# Define train and validation vectors for features
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Define the target variable
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
# Run an experiment by using Lasso regression
with mlflow.start_run():

    mlflow.set_tag("developer", "jacob")

    mlflow.log_param("train-data-path", PATH_DATA_GT_JAN)
    mlflow.log_param("valid-data-path", PATH_DATA_GT_FEB)

    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    lasso = Lasso(alpha)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle/")